In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Set working directory equal to location of current notebook file
os.getcwd()
working_directory = os.path.dirname(os.path.abspath(__vsc_ipynb_file__))
os.chdir(working_directory)
os.getcwd()

'/home/rory/Dev/AIML/practice/deep-learning-with-pytorch'

In [3]:
transform = transforms.ToTensor()

In [4]:
train_data = datasets.MNIST(root=rf'./input/cnn_data', train=True, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:00<00:00, 19.0MB/s]


Extracting ./input/cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to ./input/cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 727kB/s]


Extracting ./input/cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./input/cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:00<00:00, 7.03MB/s]


Extracting ./input/cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./input/cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 4.37MB/s]

Extracting ./input/cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./input/cnn_data/MNIST/raw



In [5]:
test_data = datasets.MNIST(root=rf'./input/cnn_data', train=False, download=True, transform=transform)

In [6]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./input/cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./input/cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()